In [1]:
import pandas as pd
import numpy as np
import dowhy
import causalml
import econml

C:\Users\kang_\.conda\envs\data_science\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [52]:
import sklearn

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
combined = pd.read_csv('combined_ca_bp_new.csv')

In [8]:
bdr = pd.read_csv('biodegradation_15.csv')

In [10]:
combined = combined[combined['Oil']==True]

In [11]:
bdr = bdr.melt(id_vars=('salinity','dispersant'),value_vars=['1','2','3'])

In [12]:
bdr

,salinity,dispersant,variable,value
0,0,False,1,0.318678
1,0,True,1,0.343368
2,5,False,1,0.427612
3,5,True,1,0.423018
4,15,False,1,0.421063
5,15,True,1,0.323758
6,35,False,1,0.404759
7,35,True,1,0.344526
8,50,False,1,0.311191
9,50,True,1,0.412678


In [13]:
combined.reset_index(inplace = True)

In [14]:
combined['bdr']=bdr['value']

In [15]:
combined.drop(columns=['Oil'],inplace = True)

In [16]:
combined.drop(columns=['index'],inplace = True)

In [32]:
#combined.to_csv('new_combined.csv')

In [17]:
combined

,Salinity_val,Dis,bp_rate,cell_abundance,bdr
0,0,False,0.139,0.105,0.318678
1,0,True,0.142,0.230,0.343368
2,5,False,0.139,0.185,0.427612
3,5,True,0.152,0.290,0.423018
4,15,False,0.149,0.199,0.421063
5,15,True,0.145,0.373,0.323758
6,35,False,0.109,0.098,0.404759
7,35,True,0.115,0.249,0.344526
8,50,False,0.089,0.063,0.311191
9,50,True,0.179,0.245,0.412678


In [43]:
causal_graph = """digraph {U[label="Unobserved Confounders"];
"Salinity_val"->"bdr";"Salinity_val"->"bp_rate";"Salinity_val"->"cell_abundance"
"Dis"->"bdr";"Dis"->"cell_abundance";"Dis"->"bp_rate";
"cell_abundance"->"bp_rate";"cell_abundance"->"bdr";
"bp_rate"->"bdr"
}"""


In [71]:
def causal_estimate(treatment,outcome):
    model = dowhy.CausalModel(data = cmbd_trans,
                        treatment = treatment,
                        outcome = outcome,
                        graph = causal_graph.replace("\n", " "))
    estimand = model.identify_effect(proceed_when_unidentifiable=True)
    estimate_li = model.estimate_effect(estimand,method_name = "backdoor.linear_regression", method_params = None)
    estimate_forest = model.estimate_effect(estimand,method_name ="backdoor.econml.ortho_forest.ContinuousTreatmentOrthoForest",method_params = {"init_params":{"n_jobs":-1},"fit_params":{}})
    return estimate_li,estimate_forest

In [72]:
def li_estimate(treatment,outcome):
    model = dowhy.CausalModel(data = cmbd_trans,
                        treatment = treatment,
                        outcome = outcome,
                        graph = causal_graph.replace("\n", " "))
    estimand = model.identify_effect(proceed_when_unidentifiable=True)
    estimate_li = model.estimate_effect(estimand,method_name = "backdoor.linear_regression", method_params = None)
    return estimate_li

In [54]:
scaler = sklearn.preprocessing.MinMaxScaler()

In [65]:
a = scaler.fit_transform(combined.loc[:,'bp_rate':'bdr'])

In [68]:
cmbd_trans = combined.copy()

In [69]:
cmbd_trans.loc[:,'bp_rate':'bdr'] = a

In [70]:
cmbd_trans

,Salinity_val,Dis,bp_rate,cell_abundance,bdr
0,0,False,0.504,0.185294,0.499968
1,0,True,0.528,0.552941,0.592865
2,5,False,0.504,0.420588,0.909829
3,5,True,0.608,0.729412,0.892547
4,15,False,0.584,0.461765,0.885189
5,15,True,0.552,0.973529,0.519082
6,35,False,0.264,0.164706,0.823847
7,35,True,0.312,0.608824,0.597221
8,50,False,0.104,0.061765,0.471801
9,50,True,0.824,0.597059,0.853640


In [47]:
param = [("Dis","bdr"),("Dis","cell_abundance"),("Dis","bp_rate"),("Salinity_val","bdr"),("Salinity_val","bp_rate"),("Salinity_val","cell_abundance")]

In [49]:
param_ca = [("cell_abundance","bdr"),("cell_abundance","bp_rate"),("bp_rate","bdr")]

In [73]:
result_1=[]
for i in param:
        result_1.append(causal_estimate(*i))

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['Dis'] on outcome ['bdr']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_identifier:Frontdoor variables for treatment and outcome:['bp_rate']
INFO:dowhy.causal_identifier:All common causes are observed. Causal effect can be identified.
INFO:dowhy.causal_identifier:All common causes are observed. Causal effect can be identified.
INFO:dowhy.causal_estimator:b: bdr~Dis+Salinity_val+Dis*Salinity_val
INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:INFO: Using EconML Estimator
C:\Users\kang_\.conda\envs\data_s

INFO:dowhy.causal_identifier:All common causes are observed. Causal effect can be identified.
INFO:dowhy.causal_identifier:All common causes are observed. Causal effect can be identified.
INFO:dowhy.causal_estimator:b: bdr~Salinity_val+Dis+Salinity_val*Dis
INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:INFO: Using EconML Estimator
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\dowhy\causal_estimators\econml.py:62: FutureWarning: The ContinuousTreatmentOrthoForest class has been renamed to DMLOrthoForest; an upcoming release will remove support for the old name
  self.estimator = estimator_class(**self.method_params["init_params"])
INFO:dowhy.causal_estimator:b: bdr~Salinity_val+Dis | Dis
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    7.5s finished
[Parallel(n_jobs=-1)]: Using back

In [74]:
result_2=[]
for i in param_ca:
        result_2.append(li_estimate(*i))

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['cell_abundance'] on outcome ['bdr']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_identifier:Frontdoor variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:b: bdr~cell_abundance+Dis+Salinity_val
INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['cell_abundance'] 

In [47]:
param = [("Dis","bdr"),("Dis","cell_abundance"),("Dis","bp_rate"),("Salinity_val","bdr"),("Salinity_val","bp_rate"),("Salinity_val","cell_abundance")]

In [49]:
param_ca = [("cell_abundance","bdr"),("cell_abundance","bp_rate"),("bp_rate","bdr")]

In [134]:
cate_li = pd.DataFrame()

In [135]:
cate_li['dis->bdr'] = result_1[0][0].conditional_estimates.values

In [136]:
cate_li['dis->ca'] = result_1[1][0].conditional_estimates.values

In [138]:
cate_li['dis->bp_rate'] = result_1[2][0].conditional_estimates.values

In [139]:
df_li = pd.DataFrame()

In [154]:
df_li['dis->bdr'] = [result_1[0][0].value]

In [155]:
df_li['dis->ca'] = [result_1[1][0].value]

In [156]:
df_li['dis->bp_rate'] = [result_1[2][0].value]

In [157]:
df_li['sal->bdr'] = [result_1[3][0].value]

In [158]:
df_li['sal->bp_rate'] = [result_1[4][0].value]

In [159]:
df_li['sal->ca'] = [result_1[5][0].value]

In [160]:
df_li['ca->bdr'] = [result_2[0].value]

In [161]:
df_li['ca->bp_rate'] = [result_2[1].value]

In [163]:
df_li['bp_rate->bdr'] = [result_2[2].value]

In [164]:
df_li

,dis->bdr,dis->ca,dis->bp_rate,sal->bdr,sal->bp_rate,sal->ca,ca->bdr,ca->bp_rate,bp_rate->bdr
0,0.033675,0.477124,0.300444,-0.003827,-0.000691,-0.002337,0.525198,0.812709,0.544232


In [170]:
df_ml=pd.DataFrame()

In [186]:
df_ml['dis->bdr'] = result_1[0][1].cate_estimates
df_ml['dis->ca'] = result_1[1][1].cate_estimates
df_ml['dis->bp_rate'] = result_1[2][1].cate_estimates
df_ml['sal->bdr'] = result_1[3][1].cate_estimates
df_ml['sal->bp_rate'] = result_1[0][1].cate_estimates
df_ml['sal->ca'] = result_1[1][1].cate_estimates

In [188]:
df_ml['Salinity_val']=combined['Salinity_val']
df_ml['Dis']=combined['Dis']

In [189]:
df_ml

,dis->bdr,dis->ca,dis->bp_rate,sal->bdr,sal->bp_rate,sal->ca,Salinity_val,Dis
0,-0.243607,0.303779,-0.125582,-0.007960,-0.243607,0.303779,0,False
1,-0.243607,0.270385,-0.122761,0.000157,-0.243607,0.270385,0,True
2,-0.203957,0.313013,-0.051162,-0.007960,-0.203957,0.313013,5,False
3,-0.203957,0.313005,-0.051162,0.000157,-0.203957,0.313005,5,True
4,-0.124659,0.395063,0.092038,-0.007960,-0.124659,0.395063,15,False
5,-0.124659,0.395063,0.092038,0.000157,-0.124659,0.395063,15,True
6,0.033938,0.516633,0.378437,-0.007960,0.033938,0.516633,35,False
7,0.033938,0.493624,0.378437,0.000157,0.033938,0.493624,35,True
8,0.152886,0.608061,0.593237,-0.007960,0.152886,0.608061,50,False
9,0.152886,0.604228,0.593237,0.000157,0.152886,0.604228,50,True


In [190]:
ml_avg = pd.DataFrame()

In [197]:
ml_avg ['dis->bdr'] = [result_1[0][1].value]
ml_avg ['dis->ca'] = [result_1[1][1].value]
ml_avg ['dis->bp_rate'] = [result_1[2][1].value]
ml_avg ['sal->bdr'] = [result_1[3][1].value]
ml_avg ['sal->bp_rate'] = [result_1[0][1].value]
ml_avg ['sal->ca'] = [result_1[1][1].value]

In [198]:
ml_avg

,dis->bdr,dis->ca,dis->bp_rate,sal->bdr,sal->bp_rate,sal->ca
0,-0.012319,0.476254,0.294826,-0.0039,-0.012319,0.476254
